* created by nov05 on 2024-12-19

In [5]:
## windows cmd to launch notepad to edit aws credential file
!notepad C:\Users\guido\.aws\credentials

In [6]:
## reset the session after updating credentials
import boto3 # type: ignore
boto3.DEFAULT_SESSION = None
import sagemaker # type: ignore
from sagemaker import get_execution_role # type: ignore
role_arn = get_execution_role()  ## get role ARN
if 'AmazonSageMaker-ExecutionRole' not in role_arn:
    ## Go to "IAM - Roles", search for "SageMaker", find the execution role.
    voclabs_role_arn = role_arn
    role_arn = "arn:aws:iam::026211625715:role/service-role/AmazonSageMaker-ExecutionRole-20241209T041445"
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()
print("AWS Region: {}".format(region))
print("Default Bucket: {}".format(bucket))
print("Role ARN: {}".format(role_arn))
print(f"Role voclabs ARN: {voclabs_role_arn}")  ## If local, Role ARN: arn:aws:iam::026211625715:role/voclabs

d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/20/24 09:32:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=605027;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=638277;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\guido\AppData\Local\sagemaker\sagemaker\config.yaml


[12/20/24 09:32:11] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=404438;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=164939;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[12/20/24 09:32:13] WARNING  Couldn't call 'get_role' to get Role ARN from role name voclabs to get ]8;id=33688;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=2825;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5902\5902]8;;\
                             Role path.                                                                            

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=777986;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=203118;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

AWS Region: us-east-1
Default Bucket: sagemaker-us-east-1-026211625715
Role ARN: arn:aws:iam::026211625715:role/service-role/AmazonSageMaker-ExecutionRole-20241209T041445
Role voclabs ARN: arn:aws:iam::026211625715:role/voclabs


In [11]:
%%time
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

class PredictorSerializer(Predictor):
    def __init__(self, *args, **kwargs):
        super(PredictorSerializer, self).__init__(*args, **kwargs)
        self.serializer = IdentitySerializer("image/jpeg")
        self.deserializer = JSONDeserializer() # To parse the response as JSON

model_location = 's3://sagemaker-us-east-1-026211625715/pytorch-training-2024-12-18-10-07-24-542/output/model.tar.gz'
pytorch_model = PyTorchModel(
    model_data=model_location, 
    role=role_arn, 
    entry_point='infernce2.py',
    py_version='py3',
    framework_version='1.4',
    predictor_cls=PredictorSerializer
)
predictor = pytorch_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.large',
    endpoint_name='p4-dog-breed-classification'
)
## 4m 4.3s

[12/20/24 10:42:02] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=862429;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=56167;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[12/20/24 10:42:05] INFO     Repacking model artifact                                                  ]8;id=252768;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model.py\model.py]8;;\:]8;id=44285;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\model.py#819\819]8;;\
                             (s3://sagemaker-us-east-1-026211625715/pytorch-training-2024-12-18-10-07-             
                             24-542/output/model.tar.gz), script artifact (None), and dependencies                 
                             ([]) into single tar.gz file located at                                               
                             s3://sagemaker-us-east-1-026211625715/pytorch-inference-2024-12-20-16-42-             
                             02-814/model.tar.gz. This may take some time depending on model size...               

[12/20/24 10:42:34] INFO     Creating model with name: pytorch-inference-2024-12-20-16-42-34-554    ]8;id=328187;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=430393;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4025\4025]8;;\

[12/20/24 10:42:35] INFO     Creating endpoint-config with name p4-dog-breed-classification         ]8;id=642818;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=442321;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5820\5820]8;;\

[12/20/24 10:42:36] INFO     Creating endpoint with name p4-dog-breed-classification                ]8;id=463060;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=658125;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#4642\4642]8;;\

------!CPU times: total: 8.33 s
Wall time: 4min 6s


In [ ]:
import boto3
## delete the endpoint model
sm_client = boto3.client('sagemaker')
endpoint_info = sm_client.describe_endpoint(EndpointName=predictor.endpoint_name)
endpoint_config_name = endpoint_info['EndpointConfigName']
endpoint_config_info = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
model_name = endpoint_config_info['ProductionVariants'][0]['ModelName']
sm_client.delete_model(ModelName=model_name)
print("🟢 Model deleted:", model_name)
## delete the endpoint
predictor.delete_endpoint()
print("🟢 Endpoint deleted:", predictor.endpoint_name)

In [13]:
import boto3
def delete_log_streams(s3_client_logs, log_group_name):
    # Retrieve log streams from the specified log group
    log_streams = s3_client_logs.describe_log_streams(logGroupName=log_group_name)
    # Iterate through each log stream and delete it
    for stream in log_streams['logStreams']:
        log_stream_name = stream['logStreamName']
        print(f"Deleting log stream: {log_stream_name}...")
        s3_client_logs.delete_log_stream(logGroupName=log_group_name, logStreamName=log_stream_name)
    print(f"🟢 All log streams deleted from log group: {log_group_name}")

s3_client_logs = boto3.client('logs')
log_group_name = "/aws/sagemaker/Endpoints/p4-dog-breed-classification"
delete_log_streams(s3_client_logs, log_group_name)

Deleting log stream: AllTraffic/i-0b15082fc2ac915d9...
Deleting log stream: AllTraffic/i-0c84121b68e040f1e...
Deleting log stream: AllTraffic/i-0fa0047e08c8e6608...
🟢 All log streams deleted from log group: /aws/sagemaker/Endpoints/p4-dog-breed-classification


---   

# **Helper code**  

In [1]:
## helper code: get lambda input field
import json
request_dict = {"url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg"}
json.dumps(request_dict)

'{"url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg"}'

In [2]:
## helper code: np.argmax(response, 1)
# Sample response (2D list)
response = [
    [0.1, 0.5, 0.4],
    [0.3, 0.2, 0.5],
    [0.7, 0.1, 0.2]
]
# Find the index of the maximum value for each row
argmax_indices = [max(range(len(row)), key=row.__getitem__) for row in response]
print(argmax_indices)

[1, 2, 0]


In [4]:
## helper code: np.argmax(response, 1)
response = [[0.12362315505743027,
 0.1924707293510437,
 -0.20855560898780823,
 0.35920414328575134,
 0.515799880027771,
 0.10468658804893494,
 -0.06848450005054474,
 0.5091232657432556,
 -0.6724733114242554,
 0.24694481492042542,
 0.38807186484336853,
 -0.0017203334718942642,
 0.1444128155708313,
 0.3559056222438812,
 0.1482323706150055,
 8.153356611728668e-05,
 0.0832664892077446,
 -0.30893537402153015,
 -0.03560059890151024,
 -0.013139759190380573,
 0.24271465837955475,
 -0.38126400113105774,
 0.016143739223480225,
 0.04776372015476227,
 -0.3131140470504761,
 0.13241298496723175,
 0.20108801126480103,
 -0.47472941875457764,
 0.1652786284685135,
 -0.05932958424091339,
 0.22977674007415771,
 0.1883726567029953,
 -0.09642750024795532,
 -0.041276972740888596,
 0.06366047263145447,
 0.4179475009441376,
 0.050274576991796494,
 0.18753167986869812,
 0.3124087154865265,
 0.20434792339801788,
 0.3538963496685028,
 0.3188058137893677,
 0.09767086803913116,
 0.2015574425458908,
 0.06633404642343521,
 0.08412200957536697,
 0.056016307324171066,
 -0.17052951455116272,
 -0.2349245250225067,
 -0.25639715790748596,
 0.2086973637342453,
 0.08721135556697845,
 -0.5352999567985535,
 -0.08533471822738647,
 0.27191051840782166,
 0.16714057326316833,
 0.6710963249206543,
 -0.12512287497520447,
 -0.22505265474319458,
 0.27227240800857544,
 -0.00902507547289133,
 0.22606226801872253,
 -0.03318323194980621,
 -0.22192427515983582,
 -0.25462859869003296,
 -0.3454451262950897,
 -0.33679768443107605,
 0.14472508430480957,
 -0.1343115270137787,
 -0.400614470243454,
 0.22343668341636658,
 -0.04466439411044121,
 0.08566736429929733,
 -0.43528684973716736,
 -0.41694146394729614,
 0.6127879023551941,
 -0.27406516671180725,
 0.06669636070728302,
 0.07389774173498154,
 -0.06984543055295944,
 0.2613086998462677,
 -0.08661563694477081,
 -0.3146919906139374,
 0.121930792927742,
 -0.529166579246521,
 -0.3005256652832031,
 0.24818851053714752,
 -0.19884799420833588,
 0.09939315915107727,
 -0.055615585297346115,
 0.060651134699583054,
 -0.11298143118619919,
 -0.6324812173843384,
 -0.26648280024528503,
 -0.13537660241127014,
 -0.16265150904655457,
 0.01313718967139721,
 -0.18079683184623718,
 0.2905445992946625,
 -0.5046991109848022,
 0.04331585019826889,
 -0.8473497629165649,
 0.28425562381744385,
 -0.0010796631686389446,
 -0.5155909061431885,
 0.22777695953845978,
 -0.07253670692443848,
 -0.3838135302066803,
 -0.3026571273803711,
 -0.2807310223579407,
 -0.16621139645576477,
 -0.08134207129478455,
 -0.393059641122818,
 -0.2946634590625763,
 0.11706878244876862,
 -0.849898099899292,
 -0.26835307478904724,
 -0.2079690843820572,
 -0.45156151056289673,
 -0.22978468239307404,
 -0.23455725610256195,
 -0.7798295617103577,
 -0.42192342877388,
 -0.6556041836738586,
 -0.4399099349975586,
 -0.09812542796134949,
 -0.16866500675678253,
 -0.4541800022125244,
 0.02175207994878292,
 -0.36741095781326294,
 -0.5499461889266968,
 -0.7415230870246887,
 -0.7355493307113647]]
argmax_indices = [max(range(len(row)), key=row.__getitem__) for row in response]
print(argmax_indices)

[56]
